# Diabetes Classification for County Health Department

<img src="images/diabetes.jpeg" alt="Drawing" style="width: 1000px;height:300px;float: left;"/>

## Overview
This project uses machine learning classification algorithms to predict whether a person has diabetes, as well as infer what are some of the most important features related to diabetes. A model built using CDC survey data from 2015 predicts whether a person has diabetes with a __ accuracy. The model determined ______ to be the most important features in predicting diabetes. This information can be used to inform guidance for future public health efforts.

## Business Understanding
The local health department is concerned with the current level of diabetes in their community. They need a tool they can deploy to help predict who in the community is diabetic or likely to become diabetic.

## Data Understanding
This project uses 2015 survey data from the CDC's Behavioral Risk Factor Surveillance System. The dataset contains information about the health and lifestyle of 253,860 people. The target variable in this dataset is a binary with values of 1 meaning a person is pre-diabetic or diabetic and 0 meaning they are not pre-diabetic or diabetic.

In [41]:
#import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.feature_selection import RFE, RFECV
import shap
from sklearn.metrics import accuracy_score, recall_score
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.pipeline import Pipeline

In [30]:
#import dataset and preview
df = pd.read_csv('data/diabetes_binary_5050split_health_indicators_BRFSS2015.csv')
df.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70692 entries, 0 to 70691
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Diabetes_binary       70692 non-null  float64
 1   HighBP                70692 non-null  float64
 2   HighChol              70692 non-null  float64
 3   CholCheck             70692 non-null  float64
 4   BMI                   70692 non-null  float64
 5   Smoker                70692 non-null  float64
 6   Stroke                70692 non-null  float64
 7   HeartDiseaseorAttack  70692 non-null  float64
 8   PhysActivity          70692 non-null  float64
 9   Fruits                70692 non-null  float64
 10  Veggies               70692 non-null  float64
 11  HvyAlcoholConsump     70692 non-null  float64
 12  AnyHealthcare         70692 non-null  float64
 13  NoDocbcCost           70692 non-null  float64
 14  GenHlth               70692 non-null  float64
 15  MentHlth           

In [32]:
df.describe()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
count,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,...,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000
mean,0.500000,0.563458,0.525703,0.975259,29.856985,0.475273,0.062171,0.147810,0.703036,0.611795,...,0.954960,0.093914,2.837082,3.752037,5.810417,0.252730,0.456997,8.584055,4.920953,5.698311
std,0.500004,0.495960,0.499342,0.155336,7.113954,0.499392,0.241468,0.354914,0.456924,0.487345,...,0.207394,0.291712,1.113565,8.155627,10.062261,0.434581,0.498151,2.852153,1.029081,2.175196
min,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,1.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,7.000000,4.000000,4.000000
50%,0.500000,1.000000,1.000000,1.000000,29.000000,0.000000,0.000000,0.000000,1.000000,1.000000,...,1.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,9.000000,5.000000,6.000000
75%,1.000000,1.000000,1.000000,1.000000,33.000000,1.000000,0.000000,0.000000,1.000000,1.000000,...,1.000000,0.000000,4.000000,2.000000,6.000000,1.000000,1.000000,11.000000,6.000000,8.000000
max,1.000000,1.000000,1.000000,1.000000,98.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,5.000000,30.000000,30.000000,1.000000,1.000000,13.000000,6.000000,8.000000


In [33]:
#class imbalance
df.Diabetes_binary.value_counts(normalize=True)

1.0    0.5
0.0    0.5
Name: Diabetes_binary, dtype: float64

## Data Preparation
Despite all the data being in numeric form, several features are actually categorical variables that need to be converted into dummy variables before modeling.

In [34]:
#convert categorical variables into dummy variables
categoricals = ['GenHlth', 'Age', 'Education', 'Income']
categorical_df = df[categoricals].astype("category")
dummies = pd.get_dummies(categorical_df , drop_first=True)

In [35]:
#drop original categorical features from df and concat with dummy variables
df.drop(categoricals, axis=1, inplace=True)
df = pd.concat([df, dummies], axis=1)

In [36]:
df.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,Education_4.0,Education_5.0,Education_6.0,Income_2.0,Income_3.0,Income_4.0,Income_5.0,Income_6.0,Income_7.0,Income_8.0
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,...,0,0,1,0,0,0,0,0,0,1
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,...,0,0,1,0,0,0,0,0,0,1
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,0,0,1,0,0,0,0,0,0,1
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,...,0,0,1,0,0,0,0,0,0,1
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,0,1,0,0,0,0,0,0,0,1


In [37]:
#Split the data into target(y) and predictor(X) variables
y = df['Diabetes_binary']
X = df.drop('Diabetes_binary', axis=1)

#split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [79]:
#scale data using StandardScaler transformer
scaler = StandardScaler()
scaler.fit(X_train)

#convert scaled data back into datframes
X_train_scaled = pd.DataFrame(scaler.transform(X_train), 
                              columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), 
                             columns=X_test.columns)

## Classification Modeling
To predict the binary diabetes target value, I will use a classification algorithm. I iterate through multiple algorithms to determine the optimal model based on accuracy and recall.

### Logitic Regression

In [89]:
logreg_baseline = LogisticRegression(fit_intercept=False)

#run cross_validation
logreg_baseline_results  = cross_validate(logreg_baseline, X_train_scaled, y_train,
                                  scoring=('accuracy', 'recall'),
                                  return_train_score=True)

average_scores(logreg_baseline_results)

fit_time: 0.13635563850402832
score_time: 0.007941818237304688
test_accuracy: 0.750070734697727
train_accuracy: 0.7504385224395136
test_recall: 0.7909390063339721
train_recall: 0.7908351920660206


#### Hyperparameter Tuning

In [87]:
logreg_param_grid = {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000],
                    'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
                    'max_iter': [10000]}

logreg_grid = GridSearchCV(estimator=logreg_baseline,
                           param_grid=logreg_param_grid,
                           scoring=['accuracy', 'recall'],
                           refit='recall',
                           cv=3,
                           n_jobs=1,
                           return_train_score=True)

logreg_grid.fit(X_train_scaled, y_train)

best_parameters = logreg_grid.best_params_

print('Grid Search found the following optimal parameters: ')
for param_name in sorted(best_parameters.keys()):
    print('%s: %r' % (param_name, best_parameters[param_name]))

Grid Search found the following optimal parameters: 
C: 10
max_iter: 10000
solver: 'newton-cg'


In [88]:
logreg_tuned = LogisticRegression(C=10, solver='newton-cg',
                                  max_iter=10000,
                                  fit_intercept=False)

#run cross_validation
logreg_tuned_results  = cross_validate(logreg_tuned, X_train_scaled, y_train,
                                  scoring=('accuracy', 'recall'),
                                  return_train_score=True)

average_scores(logreg_tuned_results)

fit_time: 0.4263707160949707
score_time: 0.00877366065979004
test_accuracy: 0.7499764271038785
train_accuracy: 0.7504149463192922
test_recall: 0.7910144566788855
train_recall: 0.7909200688113966


The hyperparameter tuning made little difference, increasing the recall score by only 0.001.

In [38]:
#funtion that displays model results
def average_scores(results):
    average_scores = {}
    for key, value in results.items():
        print(key+':', np.mean(value))

### Random Forest

In [90]:
#run baseline random forest algorithm
#instantiate random forest classifier
rf_baseline = RandomForestClassifier()

#run cross_validation
rf_baseline_results  = cross_validate(rf_baseline, X_train, y_train,
                                  scoring=('accuracy', 'recall'),
                                  return_train_score=True)

In [47]:
average_scores(rf_baseline_results)

fit_time: 3.6344985008239745
score_time: 0.2319887638092041
test_accuracy: 0.7350007149069537
train_accuracy: 0.9959259895965185
test_recall: 0.7746427636444474
train_recall: 0.9946056041295235


While the test accuracy and recall are similar to the logistic regression model, the test metrics differ substantially from the training scores indicating that the current model is overfitting to the training data.

#### Hyperparameter Tuning

In [ ]:
rf_param_grid = {'max_depth': [5, 10, 25, 50, None],
 'max_features': ['auto', 5],
 'min_samples_leaf': [1, 2, 4]}

grid_rf = GridSearchCV(rf, rf_param_grid, scoring=['accuracy', 'recall'],
                        cv=3, n_jobs=1, refit='recall', return_train_score=True)
grid_rf.fit(X_train, y_train)

best_parameters = grid_rf.best_params_

print('Grid Search found the following optimal parameters: ')
for param_name in sorted(best_parameters.keys()):
    print('%s: %r' % (param_name, best_parameters[param_name]))

In [91]:
#run baseline random forest algorithm
#instantiate random forest classifier
rf_tuned = RandomForestClassifier(max_depth=25, max_features=5, min_samples_leaf=4)

#run cross_validation
rf_tuned_results  = cross_validate(rf_tuned, X_train, y_train,
                                  scoring=('accuracy', 'recall'),
                                  return_train_score=True)

In [92]:
average_scores(rf_tuned_results)

fit_time: 2.3758739948272707
score_time: 0.1757338523864746
test_accuracy: 0.7475056524610142
train_accuracy: 0.8145429012328249
test_recall: 0.7885624876938658
train_recall: 0.851239196925024


### XGBoost

In [48]:
#instantiate a baseline XGBoost Classifier
xgb = XGBClassifier()

#run cross-validation
xgb_baseline_results  = cross_validate(xgb, X_train, y_train,
                                  scoring=('accuracy', 'recall'),
                                  return_train_score=True)

In [49]:
average_scores(xgb_baseline_results)

fit_time: 1.004561424255371
score_time: 0.031116962432861328
test_accuracy: 0.7455818198823682
train_accuracy: 0.7994162480106995
test_recall: 0.7832435692688319
train_recall: 0.8351408998441597


XGBoost has similarly high accuracy to the random forest classifier, but a lower recall.

#### Hyperparameter Tuning

In [ ]:
xgb_param_grid = {
    'learning_rate': [0.1, 0.2],
    'max_depth': [6],
    'min_child_weight': [1, 2],
    'subsample': [0.5, 0.7],
    'n_estimators': [100],
}

grid_xgb = GridSearchCV(xgb, xgb_param_grid, scoring=['accuracy', 'recall'],
                        cv=3, n_jobs=1, refit='recall',
                        return_train_score=True)
grid_xgb.fit(X_train_resampled, y_train_resampled)

best_parameters = grid_xgb.best_params_

print('Grid Search found the following optimal parameters: ')
for param_name in sorted(best_parameters.keys()):
    print('%s: %r' % (param_name, best_parameters[param_name]))

### Feature Selection

In [ ]:
selector = RFECV(forest,cv=5)
selector = selector.fit(X_train_resampled, y_train_resampled)

C:\Users\bento\anaconda3.0\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:16:46] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\bento\anaconda3.0\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:17:14] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\bento\anaconda3.0\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[16:20:16] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\bento\anaconda3.0\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:20:36] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\bento\anaconda3.0\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:20:54] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
selector.support_.shape

In [39]:
for i in range(len(X_train.columns)):
    print(X_train.columns[i],':', selector.support_[i])

HighBP : True
HighChol : True
CholCheck : True
BMI : True
Smoker : True
Stroke : True
HeartDiseaseorAttack : True
PhysActivity : True
Fruits : False
Veggies : True
HvyAlcoholConsump : True
AnyHealthcare : True
NoDocbcCost : True
MentHlth : True
PhysHlth : True
DiffWalk : True
Sex : True
GenHlth_2.0 : True
GenHlth_3.0 : True
GenHlth_4.0 : True
GenHlth_5.0 : True
Age_2.0 : True
Age_3.0 : True
Age_4.0 : True
Age_5.0 : True
Age_6.0 : True
Age_7.0 : True
Age_8.0 : True
Age_9.0 : True
Age_10.0 : True
Age_11.0 : True
Age_12.0 : True
Age_13.0 : True
Education_2.0 : False
Education_3.0 : True
Education_4.0 : True
Education_5.0 : True
Education_6.0 : True
Income_2.0 : True
Income_3.0 : True
Income_4.0 : True
Income_5.0 : True
Income_6.0 : True
Income_7.0 : True
Income_8.0 : True


In [40]:
X_train_selected = selector.transform(X_train_resampled)

In [43]:
logreg_2 = LogisticRegression(fit_intercept=False)

logreg_2_baseline_results  = cross_validate(logreg_2, X_train_selected, y_train_resampled,
                                  scoring=('accuracy', 'recall'))


In [44]:
average_scores(logreg_2_baseline_results)

fit_time: 0.9416869640350342
score_time: 0.03151559829711914
test_accuracy: 0.7408769783285575
test_recall: 0.9060897497632421


## Final Model Evaluation

In [25]:
X_final = X_train_scaled
y_final = y_train

final_model = RandomForestClassifier()
final_model.fit(X_final, y_final)

y_pred = final_model.predict(X_test_scaled)

print("Test Accuracy: ", accuracy_score(y_test, y_pred))
print("Test Recall: ", recall_score(y_test, y_pred))

Test Accuracy:  0.7372262773722628
Test Recall:  0.7736063708759955


In [24]:
y_pred = final_model.predict(X_train_scaled)

print("Test Accuracy: ", accuracy_score(y_train, y_pred))
print("Test Recall: ", recall_score(y_train, y_pred))

Test Accuracy:  0.7310020935890907
Test Recall:  0.7356529597830999


In [26]:
X_final = X_train_scaled
y_final = y_train

final_model = XGBClassifier()
final_model.fit(X_final, y_final)

y_pred = final_model.predict(X_test_scaled)

print("Test Accuracy: ", accuracy_score(y_test, y_pred))
print("Test Recall: ", recall_score(y_test, y_pred))

Test Accuracy:  0.7504668137837379
Test Recall:  0.7870307167235495


In [20]:
print("Test Accuracy: ", accuracy_score(y_test, y_pred))
print("Test Recall: ", recall_score(y_test, y_pred))

Test Accuracy:  0.139277830337433
Test Recall:  1.0


In [ ]:
# explain the model's predictions using SHAP
explainer = shap.Explainer(final_model)
shap_values = explainer(X_final)

# visualize the feature importances
shap.plots.bar(shap_values)

In [47]:
def plot_feature_importances(model):
    n_features = X_train.shape[1]
    plt.figure(figsize=(8,8))
    plt.barh(range(n_features), model.feature_importances_, align='center') 
    plt.yticks(np.arange(n_features), X_train.columns.values) 
    plt.xlabel('Feature importance')
    plt.ylabel('Feature')

In [28]:
X_final = X_train_scaled
y_final = y_train

final_model = KNeighborsClassifier()
final_model.fit(X_final, y_final)

y_pred = final_model.predict(X_test_scaled)

print("Test Accuracy: ", accuracy_score(y_test, y_pred))
print("Test Recall: ", recall_score(y_test, y_pred))

Test Accuracy:  0.7078594466134782
Test Recall:  0.722070534698521


## Conclusions

## Next Steps